In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import unicodedata

import re
from openpyxl import Workbook, load_workbook

In [19]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
# 엑셀 생성
wb = Workbook()
ws = wb.active

# 제목 적기
sub = ['식별키','목록-번호','목록-구분','목록-제목','목록-면수','목록-작성자','목록-일자','본문-일자(원본)','본문-일자','본문-내용','본문-제목','본문-부제(제목앞)','본문-부제(제목뒤)']
for kwd, j in zip(sub, list(range(1, len(sub)+1))):
    ws.cell(row=1, column=j).value = kwd

wb.save('./output/노동신문.xlsx')

In [20]:
list_base_url = 'http://www.rodong.rep.kp/ko/index.php?strPageID=SF01_01_03&strDate='
date = '2022-06-10'

In [21]:
browser.get(list_base_url+date)

### 크롤링 범위
- 목록-번호 / 목록-구분 / 목록-제목 / 목록-면 / 목록-작성자 / 목록-일자
- 본문-일자(원본) / 본문-일자 / 본문-내용 / 본문-제목 / 본문-부제(제목앞) / 본문-부제(제목뒤)

- '식별키','목록-번호','목록-구분','목록-제목','목록-면수','목록-작성자','목록-일자','본문-일자(원본)','본문-일자','본문-내용','본문-제목','본문-부제(제목앞)','본문-부제(제목뒤)'

## 뉴스 목록 크롤링

In [22]:
news_line_list = browser.find_elements(by=By.CLASS_NAME, value='ListNewsLineContainer')


In [23]:
news_line_list[34].find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW').text.strip()

'인민사랑의 력사를 길이 전하며 흐르는 락원의 강, 보통강'

In [24]:
def get_article_info(title_full) :
    try:
        if title_full.startswith('[') :
            article_type = title_full[title_full.find('[')+1 : title_full.find(']')].strip()
            article_page = re.findall('\d면', title_full)[0]
        else :
            article_type = ''
            article_page = re.findall('\d면', title_full)[0]

        article_title = title_full
        article_title = article_title.replace(article_type, "")
        article_title = article_title.replace(article_page, "")
        article_title = article_title.replace("[", "")
        article_title = article_title.replace("]", "")    
    except Exception as e:
        article_title = title_full
        article_type = ''
        article_page = ''
    
    return article_title.strip(), article_type.strip(), article_page.strip()

In [26]:
title_full = news_line_list[34].find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW').text.strip()
get_article_info(title_full)

('인민사랑의 력사를 길이 전하며 흐르는 락원의 강, 보통강', '', '')

In [27]:
get_article_info(title_full)

('인민사랑의 력사를 길이 전하며 흐르는 락원의 강, 보통강', '', '')

In [67]:
for news_line in news_line_list:
    temp_row = []
    
    line_no = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineNo').text.strip()
    line_title = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW').text.strip()
    line_writer = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineWriter').text.strip()
    line_date = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineDate').text.strip()
    
    article_title, article_type, article_page = get_article_info(line_title)

    temp_row.append(line_no)
    temp_row.append(article_type)
    temp_row.append(article_title)
    temp_row.append(article_page)
    temp_row.append(line_writer)
    temp_row.append(line_date)
    
    # 상세창 팝업
    title_popup = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW')
    title_popup_a = title_popup.find_element(by=By.TAG_NAME, value='a')
    title_popup_a.click()    
    browser.switch_to.window(browser.window_handles[1])
    
    key = browser.current_url[-15:]
    temp_row.insert(0, key)
    
    detail_date_origin = browser.find_element(by=By.CLASS_NAME, value='ArticleMenuDate').text
    temp_row.append(detail_date_origin)
    
    detail_date = key[:10]
    temp_row.append(detail_date)
    
    detail_title = ''
    detail_sub_title = ''
    detail_content = ''
    
    detail_content_list = browser.find_elements(by=By.CLASS_NAME, value='ArticleContent')

    for detail_line in detail_content_list:
        title = detail_line.find_elements(by=By.TAG_NAME, value='font')

        if title and title[0].text.strip() != '' :
            if 'bold' in title[0].get_attribute('style') :
                detail_title += title[0].text + ' '
            else :
                detail_sub_title += title[0].text + ' '
        elif detail_line.text.strip() != '' :
            detail_content += ' ' + detail_line.text
    
    temp_row.append(detail_content)
    temp_row.append(detail_title.strip())
    temp_row.append(detail_sub_title.strip())
    temp_row.append(detail_sub_title.strip())
    ws.append(temp_row)
    wb.save('./output/노동신문.xlsx')
    
    # 팝업창 close 후 객체 전환
    browser.close() 
    browser.switch_to.window(browser.window_handles[0])

In [16]:
title_popup = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW')
title_popup_a = title_popup.find_element(by=By.TAG_NAME, value='a')
title_popup_a.click()

In [51]:
browser.window_handles

['CDwindow-2FACECDB0A3970DC7739F128E93F439C',
 'CDwindow-6148C768679F565202DC9F8ABABF7B45',
 'CDwindow-13C5389092528A7700BC5A53049A293B']

In [8]:
browser.switch_to.window(browser.window_handles[-1])

In [9]:
key = browser.current_url[-15:]
key

'2022-06-01-0008'

## 뉴스 상세 크롤링

In [34]:
detail_date_origin = browser.find_element(by=By.CLASS_NAME, value='ArticleMenuDate').text
detail_date_origin

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ArticleMenuDate"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x0000000100805d60 chromedriver + 3792224
1   chromedriver                        0x000000010079c874 chromedriver + 3360884
2   chromedriver                        0x000000010049d184 chromedriver + 217476
3   chromedriver                        0x00000001004ccf5c chromedriver + 413532
4   chromedriver                        0x00000001004f5b2c chromedriver + 580396
5   chromedriver                        0x00000001004c23e4 chromedriver + 369636
6   chromedriver                        0x00000001007db234 chromedriver + 3617332
7   chromedriver                        0x00000001007df6c4 chromedriver + 3634884
8   chromedriver                        0x00000001007e3cb8 chromedriver + 3652792
9   chromedriver                        0x00000001007e015c chromedriver + 3637596


In [11]:
detail_date = key[:10]
detail_date

'2022-06-01'

In [28]:
detail_title = ''
detail_sub_title_pre = ''
detail_sub_title_next = ''
detail_content = ''

In [31]:
def get_text_type(line, detail_title, detail_content):
    font_tag = line.find_elements(by=By.TAG_NAME, value='font')
    if len(font_tag) > 0 and 'font-size' in font_tag[0].get_attribute('style'):
        if 'bold' in font_tag[0].get_attribute('style') : # bold인 것들 주제목
            return "TITLE"
        elif len(detail_title) > 0 and len(detail_content) == 0: # 제목은 나오고, 내용은 나오지 않은 경우
            return "POST_TITLE"
        elif len(detail_title) == 0 and len(detail_content) == 0: # 제목과 내용이 아직 나오지 않은 경우
            return "PRE_TITLE"
        else:
            return "CONTENT"
    else:
        return "CONTENT"

In [32]:
detail_content_list = browser.find_elements(by=By.CLASS_NAME, value='ArticleContent')

for detail_line in detail_content_list:
    line_type = get_text_type(detail_line, detail_title, detail_content)
    print(line_type, detail_line.text)
    if line_type == "TITLE":
        detail_title += detail_line.text + '\r\n'
    elif line_type == "PRE_TITLE":
        detail_sub_title_pre += detail_line.text + '\r\n' # 부제목(앞)
    elif line_type == "POST_TITLE":
        detail_sub_title_next += detail_line.text + '\r\n' # 부제목(뒤)
    elif line_type == "CONTENT":
        detail_content += detail_line.text + '\r\n' # 본문 내용
#     title = detail_line.find_elements(by=By.TAG_NAME, value='font')
    
#     if title and title[0].text.strip() != '' :
# #         print(title[0].text)
#         if 'bold' in title[0].get_attribute('style') : # bold인 것들 주제목
#             detail_title += title[0].text + '\r\n'
#         elif len(detail_title) > 0 and len(detail_content) == 0: # 제목이 이미 들어가고, 내용은 나오지 않은 경우
#             detail_sub_title_next += title[0].text + '\r\n' # 부제목(뒤)
#         elif len(detail_content) == 0:
#             detail_sub_title_pre += title[0].text + '\r\n' # 부제목(앞)
#     elif detail_line.text.strip() != '' and 'right' not in detail_line.get_attribute('style') : # 우측정렬은 글, 사진 작성자
#         detail_content += detail_line.text + '\r\n'

PRE_TITLE 인민사랑의 력사를 길이 전하며 흐르는 락원의 강, 보통강
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 
CONTENT 


In [33]:
print(detail_sub_title_pre)
print(unicodedata.normalize('NFKC', detail_title))
print(unicodedata.normalize('NFKC', detail_sub_title_next))
print(unicodedata.normalize('NFKC', detail_content))

인민사랑의 력사를 길이 전하며 흐르는 락원의 강, 보통강
































In [30]:
# 팝업창 종료 후 close
# browser.close() 
browser.switch_to.window(browser.window_handles[-1])